In [1]:
import requests
import os
import json
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv()

True

## Collect the data for the rest of 2016 of **main** cities

In [2]:
# Set the url
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Settings:
stations = [
            '74486',            # New York - John F. Kennedy Airport
            '72509',            # Boston Logan International Airport
            '72405'#,            # Washington D.C. National Airport
#            '72502',            # New Jersey - Newark Airport
#            '72202',            #'Miami International Airport'
#            '72243',            #'Houston, TX Intercontinental'
#            '72494',            #'San Francisco Airport'
#            '72793'             # Seattle-Tacoma Airport
            ]

start_dates = [
                '2016-03-01'
                ]

end_dates = [
            '2016-11-30'
            ]

headers = {
    "X-RapidAPI-Key": os.getenv('api_key'),
    "X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df_3 = pd.DataFrame([])

# Loop through all locations and all dates
for station in stations:
    for start_date, end_date in zip(start_dates, end_dates):
        querystring = {
                        "station": station,
                        "start": start_date,
                        "end": end_date,
                        "units": "metric"
                        }
        
        # Request data from url
        r = requests.get(url, headers=headers, params=querystring)

        time.sleep(5) #uncomment if you run into a query limit

        # Decode response with json decoder
        weather_temp_3 = r.json()

        # Flatten json response
        weather_temp_df_3 = pd.json_normalize(weather_temp_3,
                                             sep='_',
                                             record_path='data',
                                             record_prefix='weather_'
                                             )
        
        # Set the station code for all rows related to the current date range
        weather_temp_df_3['station'] = station

        # Concatenate dataframes
        weather_df_3 = pd.concat([weather_df_3, weather_temp_df_3], ignore_index=True)

# Print final dataset weather_df
weather_df_3

,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,2016-03-01,7.8,5.0,11.7,0.0,0.0,92.0,22.3,None,1017.0,None,74486
1,2016-03-02,8.2,-1.0,11.7,4.1,0.0,283.0,37.8,None,1006.7,None,74486
2,2016-03-03,-0.2,-2.7,2.8,0.0,0.0,266.0,19.4,None,1023.5,None,74486
3,2016-03-04,0.7,-1.0,3.9,3.3,30.0,38.0,20.5,None,1019.2,None,74486
4,2016-03-05,1.7,-1.6,5.6,0.0,0.0,314.0,14.8,None,1021.9,None,74486
...,...,...,...,...,...,...,...,...,...,...,...,...
820,2016-11-26,9.2,5.6,11.1,0.0,0.0,330.0,20.2,None,1022.5,None,72405
821,2016-11-27,7.5,3.9,12.2,0.0,0.0,NaN,9.4,None,1023.2,None,72405
822,2016-11-28,6.4,1.7,12.2,0.0,0.0,NaN,10.4,None,1021.1,None,72405
823,2016-11-29,11.9,8.9,17.2,3.0,0.0,NaN,7.9,None,1012.8,None,72405


In [3]:
weather_df_3['station'].unique()

array(['74486', '72509', '72405'], dtype=object)

## Collect the data for the rest of 2016 of the **other** cities

In [6]:
# Creating the DataFrame for the cities to compare


# Set the url
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Settings:
stations = [
#            '74486',  # New York - John F. Kennedy Airport
#            '72509',            # Boston Logan International Airport
#            '72405',            # Washington D.C. National Airport
#            '72502',            # New Jersey - Newark Airport
            '72202',            #'Miami International Airport'
            '72243',            #'Houston, TX Intercontinental'
            '72494',            #'San Francisco Airport'
            '72793'             # Seattle-Tacoma Airport
            ]

start_dates = [
                '2016-03-01'
                ]

end_dates = [
            '2016-11-30'
            ]

headers = {
    "X-RapidAPI-Key": os.getenv('api_key'),
    "X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df_4 = pd.DataFrame([])

# Loop through all locations and all dates
for station in stations:
    for start_date, end_date in zip(start_dates, end_dates):
        querystring = {
                        "station": station,
                        "start": start_date,
                        "end": end_date,
                        "units": "metric"
                        }
        
        # Request data from url
        r = requests.get(url, headers=headers, params=querystring)

        time.sleep(5) #uncomment if you run into a query limit

        # Decode response with json decoder
        weather_temp_4 = r.json()

        # Flatten json response
        weather_temp_df_4 = pd.json_normalize(weather_temp_4,
                                             sep='_',
                                             record_path='data',
                                             record_prefix='weather_'
                                             )
        
        # Set the station code for all rows related to the current date range
        weather_temp_df_4['station'] = station

        # Concatenate dataframes
        weather_df_4 = pd.concat([weather_df_4, weather_temp_df_4], ignore_index=True)

# Print final dataset weather_df
weather_df_4

,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,2016-03-01,21.1,16.7,26.7,0.0,NaN,NaN,8.3,None,1019.9,626.0,72202
1,2016-03-02,22.8,17.8,28.3,0.0,NaN,NaN,6.1,None,1018.7,647.0,72202
2,2016-03-03,23.4,20.0,27.2,0.0,NaN,147.0,9.0,None,1017.4,500.0,72202
3,2016-03-04,24.3,21.1,29.4,0.0,NaN,266.0,9.4,None,1015.5,606.0,72202
4,2016-03-05,22.6,18.3,25.6,0.3,NaN,26.0,12.6,None,1017.3,615.0,72202
...,...,...,...,...,...,...,...,...,...,...,...,...
1095,2016-11-26,9.4,6.1,10.6,9.7,0.0,83.0,10.8,None,1000.5,None,72793
1096,2016-11-27,6.9,5.6,8.3,8.1,0.0,178.0,16.2,None,1005.2,None,72793
1097,2016-11-28,7.1,4.4,10.0,0.0,0.0,167.0,10.1,None,1012.9,None,72793
1098,2016-11-29,7.6,5.0,10.6,1.0,0.0,152.0,10.1,None,1019.7,None,72793


In [7]:
weather_df_4['station'].unique()

array(['72202', '72243', '72494', '72793'], dtype=object)

**Don't let the next cell run, because I just copy/pasted the cell from the other notebook, but didn't exactly look over the code**

**And I also think we just need to do this part once!**

In [8]:
import sys
sys.path.append('../')  # Add the parent folder to the Python path

from sql_functions import get_engine  # Import the function

# Now you can use my_function() in your notebook

In [9]:
import psycopg2

schema = 'group3'
table_name = f'weather_main_cities_rest_of_2016'

engine = get_engine()

if engine!=None:
    try:
        weather_df_3.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

The weather_main_cities_rest_of_2016 table was imported successfully.


In [10]:
import psycopg2

schema = 'group3'
table_name = f'weather_other_cities_rest_of_2016'

engine = get_engine()

if engine!=None:
    try:
        weather_df_4.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

The weather_other_cities_rest_of_2016 table was imported successfully.
